## Imports

In [2]:
import duckdb
con = duckdb.connect()

## CAT

In [3]:
# con.execute("""SELECT * FROM 'category/*.parquet' limit 10;""").df()


In [4]:
cat = con.execute("""SELECT * FROM 'category/*.parquet' 
                        WHERE description LIKE '%grocer%'
                        OR description LIKE '%super%'
                        OR description LIKE '%wholesale%';""").df()
cat.head(5)

,category_id,description,category_type
0,54119904,"grocery stores, chain",SIC8
1,54119900,"grocery stores, nec",SIC8
2,51419900,"groceries, general line, nec",SIC8
3,54110101,"supermarkets, chains",SIC8
4,51990305,"periodicals, wholesale",SIC8


In [5]:
con.execute("""SELECT COUNT(*) FROM cat;""").df()

,count_star()
0,110


## DMA

In [6]:
# con.execute("""SELECT * FROM 'dma/*.parquet' limit 10;""").df()

In [7]:
# dma = con.execute("""SELECT * FROM 'dma/*.parquet' WHERE dma_name LIKE '%CA%';""").df()
dma = con.execute("""SELECT * FROM 'dma/*.parquet' WHERE dma_name='San Francisco-Oakland-San Jose, CA';""").df()
dma

,dma_code,dma_name
0,807,"San Francisco-Oakland-San Jose, CA"


In [8]:
con.execute("""SELECT COUNT(*) FROM dma;""").df()

,count_star()
0,1


## Metric

In [9]:
# con.execute("""SELECT * FROM 'metric/*.parquet' limit 10;""").df()

In [10]:
# con.execute("""SELECT COUNT(*) FROM 'metric/*.parquet' WHERE category='cross_shopping'""").df()


In [11]:
# met = con.execute("""SELECT * FROM 'metric/*.parquet' WHERE category='cross_shopping' and sub_category = 'daily'""").df()
met = con.execute("""CREATE TEMP TABLE met AS SELECT * FROM 'metric/*.parquet' WHERE category='cross_shopping' and sub_category = 'weekly'""")

In [12]:
con.execute("""SELECT COUNT(*) FROM met;""").df()

,count_star()
0,34146733


## Place

In [13]:
# con.execute("""SELECT * FROM 'place/*.parquet' limit 10;""").df()

In [14]:
# place = con.execute("""SELECT * FROM 'place/*.parquet' WHERE names LIKE '%Whole Food%';""").df()
place = con.execute("""CREATE TEMP TABLE plc AS SELECT * FROM 'place/*.parquet' WHERE names LIKE '%Whole Food%';""")

In [15]:
place.df()

,Count
0,794


In [16]:
con.execute("""SELECT COUNT(*) FROM plc;""").df()

,count_star()
0,794


## Pre-join EDA

## Join df's

In [17]:
join_df = con.execute("""SELECT * FROM plc 
                            --JOIN cat ON plc.category_ids=cat.category_id
                            JOIN dma ON plc.dma_id=dma.dma_code
                            JOIN met ON plc.spatial_id=met.spatial_id;""").df()

In [20]:
# con.execute("""SELECT COUNT(*) FROM join_df;""")
g_df = con.execute("""SELECT metric, SUM(value) FROM join_df GROUP BY metric ORDER BY metric DESC;""").df()

In [21]:
g_df.to_csv('g_df.csv')